# 일상/연애 주제의 한국어 대화 이진 분류 모델
# 참고자료: https://velog.io/@seolini43/%EC%9D%BC%EC%83%81%EC%97%B0%EC%95%A0-%EC%A3%BC%EC%A0%9C%EC%9D%98-%ED%95%9C%EA%B5%AD%EC%96%B4-%EB%8C%80%ED%99%94-BERT%EB%A1%9C-%EC%9D%B4%EC%A7%84-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0%ED%8C%8C%EC%9D%B4%EC%8D%ACColab-%EC%BD%94%EB%93%9C

In [1]:
!pip install transformers

In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [3]:
# GPU 맞게 설정되었는지 확인하느 코드
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [4]:
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/NLP/Classification_BERT")
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/Colab Notebooks/NLP/Classification_BERT'

In [51]:
# 데이터 불러오기
chatbot_data = pd.read_csv('ChatbotData.csv')

# 데이터 정보
## 11,876개의 한글 대화 문답으로 되어 있는 인공데이터로,  일상 대화, 이별과 관련된 대화, 긍정적인 사랑에 대한 대화가 각각 0, 1, 2로 라벨링 되어 있다.
## 컬럼 설명
### Q: 한글 질의
### A: 한글 답변
### label: 일상 대화 - 0, 연애 관련 이별(부정) 대화 - 1, 연애 관련 사랑(긍정) 대화 - 2

In [52]:
chatbot_data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [53]:
chatbot_data.loc[(chatbot_data['label'] == 2), 'label'] = 1

# train-test set 나누기
## 데이터 확인해보면 0~5290까지의 행이 모두 클래스가 0인 행이고, 5291행부터 마지막 행까지 모두 클래스가 1임.
## 1. 특정 비율로 데이터셋을 나눌 때 클래스가 0인 데이터와 1인 데이터 골고루 섞기 위해 데이터 랜덤으로 섞은 뒤 나누는 작업 진행
## 2. 그냥 평상시처럼 train_test_split 함수 사용해서 dataset 나누기

In [54]:
# 1.
chatbot_data_shuffled = chatbot_data.sample(frac=1).reset_index(drop=True)

In [55]:
#train data & test data 로드
train = chatbot_data_shuffled[:9000]
test = chatbot_data_shuffled[9000:]

In [56]:
train['label'].value_counts()

1    4973
0    4027
Name: label, dtype: int64

In [57]:
test['label'].value_counts()

1    1560
0    1263
Name: label, dtype: int64

In [12]:
# 2.
X_train, X_test = train_test_split(chatbot_data, test_size = 0.2 , random_state = 42)
X_train

,Q,A,label
10512,엄청 로맨틱해,생각만해도 달콤하네요.,1
1199,대리님이 너무 갈궈,더 웃으면서 대해보세요.,0
9841,사내커플인데 비밀연애임. 답답해.,비밀연애가 말도 못하고 힘들죠.,1
5595,그 사람이 참 그리워,사랑했나봐요.,1
7228,왜,궁금하네요.,1
...,...,...,...
11284,좋아했지만 고백은 못하겠어.,애틋한 사랑이네요.,1
5191,화장실!!,화장실 가세요.,0
5390,6개월이 지나도 왜이런거죠?,물리적 시간에 비례하지 않으니까요.,1
860,내가 제일 문제인 듯,당신은 하나밖에 없는 소중한 사람이에요.,0


In [13]:
X_train['label'].value_counts()

1    5256
0    4202
Name: label, dtype: int64

In [14]:
X_test['label'].value_counts()

1    1277
0    1088
Name: label, dtype: int64

# Train set 전처리
## 1. BERT 분류 모델의 경우 각 문장의 앞마다 [CLS]를 붙여 인식시키고, 문장의 종료는 [SEP]를 붙여 인식시킨다. [CLS]을 인식함으로써 문장의 처음이라 알 수 있게 하고, [SEP]을 인식함으로써 문장의 끝을 알 수 있게 하기 위함이다.
## 2. BERT에서 사용하는 토크나이저인 WordPiece: 단어를 토큰화할 때, 단어집합에 없는 단어는 더 쪼개서 ##을 붙여주는 방식
## 3. 문장의 최대 시퀀스를 설정하여 정수 인코딩과 제로 패딩을 수행
## 4. Attention Mask: 0 값을 가지는 패딩 토큰에 대해서 어텐션 연산을 불필요하게 수행하지 않도록 단어와 패딩 토큰을 구분할 수 있게 알려주는 것을 말한다. 따라서 [40311, 9435, 102, 0, 0]와 같은 패딩된 데이터가 있을 때, 패딩된 값은 '0', 패딩되지 않은 단어는 '1'의 값을 갖도록 시리얼 데이터를 만들어 주어야 한다.

In [58]:
# 1.
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.Q]
sentences[:5]

['[CLS] 역류성 식도염 온 거 같아. [SEP]',
 '[CLS] 원피스 입는 게 낫겠지 [SEP]',
 '[CLS] 보고싶어 사무칠때 [SEP]',
 '[CLS] 주식이 떨어졌어 [SEP]',
 '[CLS] 과식했나 봐 [SEP]']

In [59]:
# 0과 1 라벨이 담겨 있는 컬럼을 array로 따로 저장
labels = train['label'].values
labels

array([0, 0, 1, ..., 1, 1, 0])

In [60]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('안녕하세요')
print(result)
'''현재 단어 집합이 비어있는 상태이므로 '안녕하세요'라는 단어가 단어집합에 없다.
따라서 이 단어는 한번 더 쪼개어 지는데 '##'을 붙임으로써 '##녕', '##하', '##세', '##요' 가 어떠한 단어의 서브워드라는 것을 나타내준다.
이렇게 나타내 줌으로써 단어의 의미를 잃지 않고 다시 원래 단어로 복원을 할 수 있게 된다.'''

['안', '##녕', '##하', '##세', '##요']


"현재 단어 집합이 비어있는 상태이므로 '안녕하세요'라는 단어가 단어집합에 없다.\n따라서 이 단어는 한번 더 쪼개어 지는데 '##'을 붙임으로써 '##녕', '##하', '##세', '##요' 가 어떠한 단어의 서브워드라는 것을 나타내준다.\n이렇게 나타내 줌으로써 단어의 의미를 잃지 않고 다시 원래 단어로 복원을 할 수 있게 된다."

In [61]:
# 2.
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

In [62]:
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] 역류성 식도염 온 거 같아. [SEP]
['[CLS]', '역', '##류', '##성', '식', '##도', '##염', '온', '거', '같', '##아', '.', '[SEP]']


In [63]:
# 3.
MAX_LEN = 128 #최대 시퀀스 길이 설정
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([   101,   9566,  46520,  17138,   9486,  12092, 119144,   9582,
         8863,   8855,  16985,    119,    102,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [64]:
# 4.
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# train set을 test set과 validation set으로 분리하기

In [65]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels,
                                                                                    random_state=2000,
                                                                                    test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state=2000,
                                                       test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [66]:
batch_size = 32
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [67]:
train_inputs.shape

torch.Size([8100, 128])

In [68]:
train_labels.shape

torch.Size([8100])

In [69]:
train_masks.shape

torch.Size([8100, 128])

# Test set 전처리

In [70]:
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in test.Q]

# 라벨 데이터
test_labels = test['label'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(attention_masks)



# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [71]:
test_inputs.shape

torch.Size([2823, 128])

In [72]:
test_labels.shape

torch.Size([2823])

In [73]:
test_masks.shape

torch.Size([2823, 128])